<a href="https://colab.research.google.com/github/unburied/DS-Unit-2-Linear-Models/blob/master/Doing_Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Get/Split/Model(basic)


In [0]:
import pandas as pd
import numpy as np

In [0]:
WEB = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/nyc/nyc-rent-2016.csv'

df = pd.read_csv(WEB)

In [4]:
#View Columns
df.head().transpose()

,0,1,2,3,4
bathrooms,1.5,1,1,1,1
bedrooms,3,2,1,1,4
created,2016-06-24 07:54:24,2016-06-12 12:19:27,2016-04-17 03:26:41,2016-04-18 02:22:02,2016-04-28 01:32:41
description,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,,"Top Top West Village location, beautiful Pre-w...",Building Amenities - Garage - Garden - fitness...,Beautifully renovated 3 bedroom flex 4 bedroom...
display_address,Metropolitan Avenue,Columbus Avenue,W 13 Street,East 49th Street,West 143rd Street
latitude,40.7145,40.7947,40.7388,40.7539,40.8241
longitude,-73.9425,-73.9667,-74.0018,-73.9677,-73.9493
price,3000,5465,2850,3275,3350
street_address,792 Metropolitan Avenue,808 Columbus Avenue,241 W 13 Street,333 East 49th Street,500 West 143rd Street
interest_level,medium,low,high,low,low


In [5]:
#convert time string object to datetime object and set as index
df['created'] = pd.to_datetime(df.created, infer_datetime_format= True)

df = df.set_index('created')
df.head()

,bathrooms,bedrooms,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,exclusive,terrace,loft,garden_patio,common_outdoor_space,wheelchair_access
created,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-06-24 07:54:24,1.5,3,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-06-12 12:19:27,1.0,2,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-04-17 03:26:41,1.0,1,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-04-18 02:22:02,1.0,1,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-04-28 01:32:41,1.0,4,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
#Split Data Based on date
train = df[df.index.month < 6]
test = df[df.index.month == 6]

train.shape, test.shape

((31515, 33), (16785, 33))

In [7]:
#extract target values from df
y_train = train.pop('price')
y_test = test.pop('price')

y_train.shape, y_test.shape

((31515,), (16785,))

In [0]:
#get a baseline model
baseline = np.full_like(y_test, y_train.mean())

##Baseline

In [9]:
#get metric to test baseline 
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_test, baseline)

1052.5193327375632

##Upgrade Baseline Model


In [0]:
#import and instantiate model
from sklearn.linear_model import LinearRegression

model = LinearRegression()

In [0]:
#select features to model
#here we will start with all numerical features and exclude any objects
features = list(train.dtypes[train.dtypes != 'object'].index)

X_train = train[features]
X_test = test[features]

In [0]:
#Train model and store result
#Will be updating X_train and X_test so dont need to pass as new values 
def score(train,test):
  model.fit(train, y_train)
  y_pred = model.predict(test)

  return mean_absolute_error(y_test, y_pred)
  

In [13]:
score(X_train, X_test)

618.6796864572029

##Upgraded Baseline

In [14]:
better_baseline = score(X_train, X_test)
print(better_baseline)

618.6796864572029


##Feature Selection
  

###Numerical Features


In [15]:
#Iterate over features to determine which have the highest impact

impact = {}

for feature in features:
  model.fit(train[[feature]], y_train)
  y_pred = model.predict(test[[feature]])
  impact[feature] = mean_absolute_error(y_test, y_pred)
                
print(impact)

{'bathrooms': 826.8186291735686, 'bedrooms': 865.7153554398519, 'latitude': 1048.5914341618604, 'longitude': 989.0487004130522, 'elevator': 1021.1914238867432, 'cats_allowed': 1049.6548133279234, 'hardwood_floors': 1050.474902267552, 'dogs_allowed': 1049.2944534089013, 'doorman': 1002.990620652015, 'dishwasher': 1026.0537865120189, 'no_fee': 1044.9120965283466, 'laundry_in_building': 1052.8562017778665, 'fitness_center': 1020.8674997057973, 'pre-war': 1052.1651020895426, 'laundry_in_unit': 1028.7246629448905, 'roof_deck': 1044.951424200772, 'outdoor_space': 1038.9376029999073, 'dining_room': 1029.926584738313, 'high_speed_internet': 1047.8368618041293, 'balcony': 1040.796298235273, 'swimming_pool': 1046.0479246061477, 'new_construction': 1052.4032354966685, 'exclusive': 1052.262266759364, 'terrace': 1042.9198257172632, 'loft': 1051.8527496972877, 'garden_patio': 1046.3402013438135, 'common_outdoor_space': 1051.7843743981514, 'wheelchair_access': 1051.120211011886}


In [16]:
#structure as a series and sort. extracting top 5 values

impact_table = pd.Series(impact)
impact_table = impact_table.sort_values()

impact_table.head()

bathrooms          826.818629
bedrooms           865.715355
longitude          989.048700
doorman           1002.990621
fitness_center    1020.867500
dtype: float64

In [17]:
#Get number of effective features that matches or beats better_baseline

index = 0 
extract = {}
prev_mae = 0

for index in range(1,len(impact_table)):
  features = impact_table.index[0:index]
  
  if index == 0:
      model.fit(train[[features]], y_train)
      y_pred = model.predict(test[[features]])
      mae = mean_absolute_error(y_test, y_pred)
      print(mae)
  else:
      model.fit(train[features], y_train)
      y_pred = model.predict(test[features])
      mae = mean_absolute_error(y_test, y_pred)
      print(mae)
      
  #Find features whose effect on score is greater than 1
  effect = abs(mae - prev_mae)
  if effect > 1:
    extract[impact_table.index[index]] = effect          
    prev_mae = mae
    
print(extract)

826.8186291735686
754.5508722934989
667.2933147298353
635.1528773589849
635.122921947556
634.3752688719886
634.0690256373571
632.0777436135802
632.0763605880798
632.3628533827904
632.0742813294919
631.9958764459212
630.4861046080734
627.4049730300144
627.39554671891
627.4322194886214
624.304761538569
625.1484309228863
624.2135689536922
624.2104961858942
621.0487661651432
621.0629038370165
620.7512149718334
620.7676930359772
620.793134769754
620.8129165225054
620.1255609377877
{'bedrooms': 826.8186291735686, 'longitude': 72.26775688006978, 'doorman': 87.25755756366357, 'fitness_center': 32.14043737085035, 'laundry_in_unit': 1.083851721627866, 'dining_room': 1.9912820237768756, 'roof_deck': 1.5916390055067495, 'swimming_pool': 3.0811315780590576, 'latitude': 3.100211491445407, 'wheelchair_access': 3.255995373425776}


Not better than our previous baseline. We ended up with an empty dict because no subset of features scored better than the entire set of numberical features. But from this we can conclude the top 4 features offered the most significant boost to the score, so we can use those. Lets see what they are.

In [18]:
impact_table.head(4)

bathrooms     826.818629
bedrooms      865.715355
longitude     989.048700
doorman      1002.990621
dtype: float64

These features make sense. The number of bed/baths significantly affects the value of a residence. There may even be a correalation there with respect to square footage, of which is a key indicator in pricing. Doorman is a luxary accomodation, so will aslo affect pricing. And longitude will determine which borrow the appartment is listed in. From east to west there is Staten Island, Manhattan, Brooklyn, The Bronx, and Queens. These boroughs appear to be more affected by this metric than latitude.  From the output above, we can already determine the score to be $635.15

###Non-numericals

In [19]:
#extract and look at objects in df
features = list(df.dtypes[df.dtypes == 'object'].index)
features

['description', 'display_address', 'street_address', 'interest_level']

In [20]:
df[features].head()

,description,display_address,street_address,interest_level
created,,,,
2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,792 Metropolitan Avenue,medium
2016-06-12 12:19:27,,Columbus Avenue,808 Columbus Avenue,low
2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,241 W 13 Street,high
2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,333 East 49th Street,low
2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,500 West 143rd Street,low


It looks like interest_level is the only feature worth using for a simple boost to our model.

In [0]:
#Encode interest_level to model
interest_train = pd.get_dummies(train.interest_level)
interest_test = pd.get_dummies(test.interest_level)

In [22]:
score(interest_train, interest_test)

1028.1011245159368

Not a huge boost. Lets see what it looks like when we add it to our 4 numerical features

###Combinations

In [0]:
#Merge encoded interest_level data with top 4 numerical features
top_4 = impact_table.index[0:5]

X_train = pd.concat([train[top_4], interest_train], axis = 1)
X_test = pd.concat([test[top_4], interest_test], axis = 1)

In [24]:
X_train.head()

,bathrooms,bedrooms,longitude,doorman,fitness_center,high,low,medium
created,,,,,,,,
2016-04-17 03:26:41,1.0,1,-74.0018,0,0,1,0,0
2016-04-18 02:22:02,1.0,1,-73.9677,0,0,0,1,0
2016-04-28 01:32:41,1.0,4,-73.9493,0,0,0,1,0
2016-04-19 04:24:47,2.0,4,-74.0028,0,0,0,0,1
2016-04-27 03:19:56,1.0,2,-73.9660,0,0,0,1,0


In [0]:
new_best = score(X_train,X_test)

That is a pretty significant boost! With just 5(technically 7) features, we were able to bring the MAE down to 606.17. I assume that if we used all the numerical features AND the encoded interest level, we would see another drop of about 10.

## Updated Score

In [26]:
print(new_best)

606.172802824469


##All Features

In [27]:
#Merge all features together and test

features = list(df.dtypes[df.dtypes != 'object'].index)
features.remove('price')

X_train = pd.concat([train[features], interest_train], axis = 1)
X_test = pd.concat([test[features], interest_test], axis =1)

score(X_train, X_test)

592.271534074256

With all the numerical features, and the dummy features from interest_level, we are able to bring the score down to 592. Our best yet!

##Feature Trimming


We can start by looking at our extract disctionary from previous sections to get an idea of where to start

In [28]:
extract = pd.Series(extract).transpose().sort_values(ascending = False)
print(extract)

bedrooms             826.818629
doorman               87.257558
longitude             72.267757
fitness_center        32.140437
wheelchair_access      3.255995
latitude               3.100211
swimming_pool          3.081132
dining_room            1.991282
roof_deck              1.591639
laundry_in_unit        1.083852
dtype: float64


In [29]:
#Switched to little x here because the score is low
features = extract.index.to_list()

x_train = pd.concat([train[features], interest_train], axis = 1)
x_test = pd.concat([test[features], interest_test], axis =1)

score(x_train, x_test)

681.1410401417156

It looks like when we try to extract what appear to be primary features, effect isnt as impactful as one would expect. 

##Feature Engineering

###Description Counts


In [30]:
#Lets look at description to engineer some features
df.description.sample(5)

created
2016-06-01 02:12:12    Currently being gut renovated, this apartment ...
2016-04-08 06:00:33    Elevator, Laundry, Subway, Balcony, Dining Roo...
2016-06-08 03:50:29    Mega home here in the East Village - only 2 fl...
2016-06-08 06:35:46    This is a gorgeous Pre-war 2Br on Prime Upper ...
2016-05-14 05:52:39    Spacious studio with lots of light, high ceili...
Name: description, dtype: object

In [31]:
df.description.isna().sum()

1421

In [0]:
df.description.fillna(value = ' ', inplace = True)

In [33]:
#Lets see how many words are in each description
df['description_count'] = df.description.str.count('\w')

df.description_count.describe()

count    48300.000000
mean       469.296190
std        310.682372
min          0.000000
25%        263.000000
50%        437.000000
75%        631.000000
max       3651.000000
Name: description_count, dtype: float64

In [0]:
#What does this do for our model?

train = df[df.index.month < 6]
test = df[df.index.month == 6]

y_train = train.pop('price')
y_test = test.pop('price')

In [35]:
feature = 'description_count'

model.fit(train[[feature]], y_train)
y_pred = model.predict(test[[feature]])

print(mean_absolute_error(y_test,y_pred))

1037.2012769558742


Doesnt look all that great. Lets combine with some of our other features to see how it stands. 

In [36]:
X_train = pd.concat([X_train, train.description_count], axis = 1)
X_test = pd.concat([X_test, test.description_count], axis = 1)

score(X_train, X_test)

592.2754297365971

Looks like that was no help at all! Means that its not HOW much was said, but WHAT!

###NLP

Vectorize tokens of the description data


In [37]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

class LemmaTokenizer(object):
  def __init__(self):
    self.wnl = WordNetLemmatizer()
  def __call__(self, description):
    return [self.wnl.lemmatize(t) for t in nltk.word_tokenize(description)]

tfidf = TfidfVectorizer(tokenizer = LemmaTokenizer(), lowercase = True)

X = tfidf.fit_transform(df.description)

In [0]:
words = pd.DataFrame(data = X.toarray(), columns = tfidf.get_feature_names()).round(3)

In [1]:
X

NameError: ignored